In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Tue Nov  5 07:39:45 PST 2019
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.6 LTS
Release:	16.04
Codename:	xenial

------------
HOSTNAME: 
swoose

------------
Computer Specs:

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    2
Core(s) per socket:    6
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 44
Model name:            Intel(R) Xeon(R) CPU           X5670  @ 2.93GHz
Stepping:              2
CPU MHz:               2925.971
BogoMIPS:              5851.97
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              12288K
NUMA node0 CPU(s):     0-23
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr

No LSB modules are available.


### Set variables
`%env` variables are good for passing to bash cells

In [2]:
# Set workding directory
%env wd=/home/sam/analyses/20191105_swoose_pgen_v074_renaming
wd="/home/sam/analyses/20191105_swoose_pgen_v074_renaming"

%env rsync_owl=owl:/volume1/web/halfshell/genomic-databank/
%env wget_command=--directory-prefix=${wd} --quiety --no-directories --no-check-certificate  https://owl.fish.washington.edu/halfshell/genomic-databank/

%env new_prefix=Panopea-generosa-v1.0
%env fa_prefix=Pgenerosa_v074
%env gff_prefix=Panopea-generosa-vv0.74

env: wd=/home/sam/analyses/20191105_swoose_pgen_v074_renaming
env: rsync_owl=owl:/volume1/web/halfshell/genomic-databank/
env: files_list=Panopea-generosa-vv0.74.a4.CDS.gff3 Panopea-generosa-vv0.74.a4.exon.gff3 Panopea-generosa-vv0.74.a4.gene.gff3 Panopea-generosa-vv0.74.a4.mRNA.gff3 Panopea-generosa-vv0.74.a4.repeat_region.gff3 Panopea-generosa-vv0.74.a4.rRNA.gff3 Panopea-generosa-vv0.74.a4.tRNA.gff3 Pgenerosa_v074.fa Pgenerosa_v074.fa.fai Pgenerosa_v074.CpG.gff Panopea-generosa-vv0.74.a4.intergenic.bed  Panopea-generosa-vv0.74.a4.introns.bed Panopea-generosa-vv0.74.a4.repeats.DNA.gff3 Panopea-generosa-vv0.74.a4.repeats.LTR.gff3 Panopea-generosa-vv0.74.a4.repeats.SINE.gff3 Panopea-generosa-vv0.74.a4.repeats.Unknown.gff3 Panopea-generosa-vv0.74.a4.repeats.LINE.gff3 Panopea-generosa-vv0.74.a4.repeats.RC.gff3 Panopea-generosa-vv0.74.a4.repeats.Simple_repeat.gff3
env: wget_command=--directory-prefix=$/home/sam/analyses/20191105_swoose_pgen_v074_renaming --quiety --no-directories --no-chec

#### Create necessary directories

In [3]:
%%bash
mkdir --parents ${wd}

In [4]:
cd {wd}

/home/sam/analyses/20191105_swoose_pgen_v074_renaming


#### Download Pgen_v074 files


In [16]:
%%bash
# Create array of files from list
files_array=(Panopea-generosa-vv0.74.a4.CDS.gff3 Panopea-generosa-vv0.74.a4.exon.gff3 Panopea-generosa-vv0.74.a4.gene.gff3 Panopea-generosa-vv0.74.a4.mRNA.gff3 Panopea-generosa-vv0.74.a4.repeat_region.gff3 Panopea-generosa-vv0.74.a4.rRNA.gff3 Panopea-generosa-vv0.74.a4.tRNA.gff3 Pgenerosa_v074.fa Pgenerosa_v074.fa.fai Pgenerosa_v074.CpG.gff Panopea-generosa-vv0.74.a4.intergenic.bed Panopea-generosa-vv0.74.a4.introns.bed Panopea-generosa-vv0.74.a4.repeats.DNA.gff3 Panopea-generosa-vv0.74.a4.repeats.LTR.gff3 Panopea-generosa-vv0.74.a4.repeats.SINE.gff3 Panopea-generosa-vv0.74.a4.repeats.Unknown.gff3 Panopea-generosa-vv0.74.a4.repeats.LINE.gff3 Panopea-generosa-vv0.74.a4.repeats.RC.gff3 Panopea-generosa-vv0.74.a4.repeats.Simple_repeat.gff3)


for file in "${files_array[@]}"
do
  rsync \
  --archive \
  --progress \
  --verbose \
  "${rsync_owl}${file}" \
  .
done

ls -lh

receiving incremental file list
Panopea-generosa-vv0.74.a4.CDS.gff3
     62,862,005 100%   27.44MB/s    0:00:02 (xfr#1, to-chk=0/1)

sent 30 bytes  received 62,869,800 bytes  9,672,281.54 bytes/sec
total size is 62,862,005  speedup is 1.00
receiving incremental file list
Panopea-generosa-vv0.74.a4.exon.gff3
     64,663,603 100%   26.86MB/s    0:00:02 (xfr#1, to-chk=0/1)

sent 30 bytes  received 64,671,619 bytes  9,949,484.46 bytes/sec
total size is 64,663,603  speedup is 1.00
receiving incremental file list
Panopea-generosa-vv0.74.a4.gene.gff3
     10,997,681 100%   25.90MB/s    0:00:00 (xfr#1, to-chk=0/1)

sent 30 bytes  received 10,999,145 bytes  3,142,621.43 bytes/sec
total size is 10,997,681  speedup is 1.00
receiving incremental file list
Panopea-generosa-vv0.74.a4.mRNA.gff3
     10,666,540 100%   22.46MB/s    0:00:00 (xfr#1, to-chk=0/1)

sent 30 bytes  received 10,667,964 bytes  7,111,996.00 bytes/sec
total size is 10,666,540  speedup is 1.00
receiving incremental file list
Panop

#### If need to download via wget, uncomment lines in the cell below

In [7]:
# %%bash
# time \
# wget "${wget_gffs}"
# wget "${wget_fasta}"
# ls -lh ${wd}

### Change scaffold names and file names

In [18]:
%%bash

files_array=(Panopea-generosa-vv0.74.a4.CDS.gff3 Panopea-generosa-vv0.74.a4.exon.gff3 Panopea-generosa-vv0.74.a4.gene.gff3 Panopea-generosa-vv0.74.a4.mRNA.gff3 Panopea-generosa-vv0.74.a4.repeat_region.gff3 Panopea-generosa-vv0.74.a4.rRNA.gff3 Panopea-generosa-vv0.74.a4.tRNA.gff3 Pgenerosa_v074.fa Pgenerosa_v074.fa.fai Pgenerosa_v074.CpG.gff Panopea-generosa-vv0.74.a4.intergenic.bed Panopea-generosa-vv0.74.a4.introns.bed Panopea-generosa-vv0.74.a4.repeats.DNA.gff3 Panopea-generosa-vv0.74.a4.repeats.LTR.gff3 Panopea-generosa-vv0.74.a4.repeats.SINE.gff3 Panopea-generosa-vv0.74.a4.repeats.Unknown.gff3 Panopea-generosa-vv0.74.a4.repeats.LINE.gff3 Panopea-generosa-vv0.74.a4.repeats.RC.gff3 Panopea-generosa-vv0.74.a4.repeats.Simple_repeat.gff3)

# Array of old scaffold names
# Uses first column of FastA index file to get scaffold names
mapfile -t orig_scaffold_names < <(awk '{print $1}' Pgenerosa_v074.fa.fai)

# Array of new scaffold names
mapfile -t new_scaffold_names < <(for number in {01..18}; do echo "Scaffold_${number}"; done)


for file in "${files_array[@]}"
do
  # Set new filename, depending if FastA or GFF
  name_check="${file%%.*}"
  if [ "${name_check}" = "${fa_prefix}" ]
  then
    new_name=${file/$fa_prefix/$new_prefix}
  else
    new_name=${file/$gff_prefix/$new_prefix}
  fi
  # sed substitution
  # creates sed script to find original scaffold names and replace them with new scafold names
  # and passes to sed via stdin
  for index in "${!orig_scaffold_names[@]}"
  do
    printf "s/%s/%s/\n" "${orig_scaffold_names[index]}" "${new_scaffold_names[index]}"
  done | sed --file - "${file}" \
  >> "${new_name}"
done

ls -ltrh

total 7.9G
-rw-rw-rw- 1 sam users 914M Jun 26 08:49 Pgenerosa_v074.fa
-rw-rw-rw- 1 sam users 1.3K Jun 26 08:54 Pgenerosa_v074.fa.fai
-rw-rw-r-- 1 sam users 2.5G Aug 21 11:45 Pgenerosa_v074.CpG.gff
-rwxr--r-- 1 sam users  60M Oct 14 10:13 Panopea-generosa-vv0.74.a4.CDS.gff3
-rwxr--r-- 1 sam users  62M Oct 14 10:13 Panopea-generosa-vv0.74.a4.exon.gff3
-rwxr--r-- 1 sam users  11M Oct 14 10:13 Panopea-generosa-vv0.74.a4.gene.gff3
-rwxr--r-- 1 sam users  11M Oct 14 10:13 Panopea-generosa-vv0.74.a4.mRNA.gff3
-rwxr--r-- 1 sam users 373M Oct 14 10:13 Panopea-generosa-vv0.74.a4.repeat_region.gff3
-rwxr--r-- 1 sam users 1.5K Oct 14 10:13 Panopea-generosa-vv0.74.a4.rRNA.gff3
-rwxr--r-- 1 sam users 2.8M Oct 14 10:13 Panopea-generosa-vv0.74.a4.tRNA.gff3
-rw-rw-r-- 1 sam users  11M Oct 29 08:52 Panopea-generosa-vv0.74.a4.repeats.SINE.gff3
-rw-rw-r-- 1 sam users 4.5M Oct 29 08:52 Panopea-generosa-vv0.74.a4.repeats.Simple_repeat.gff3
-rw-rw-r-- 1 sam users 140K Oct 29 08:52 Panopea-generosa-vv0.74.a4.

In [19]:
%%bash
head Panopea-generosa-v1.0.a4.CDS.gff3

echo ""
echo "--------------------"
awk '{print $1}' Panopea-generosa-v1.0.a4.CDS.gff3 | sort | uniq

##gff-version 3
##Generated using GenSAS, Monday 7th of October 2019 04:54:37 AM
##Project Name : Pgenerosa_v074
Scaffold_01	GenSAS_5d9637f372b5d-publish	CDS	2	125	.	+	0	ID=PGEN_.00g000010.m01.CDS01;Name=PGEN_.00g000010.m01.CDS01;Parent=PGEN_.00g000010.m01;original_ID=cds.21510-PGEN_.00g234140.m01;Alias=cds.21510-PGEN_.00g234140.m01
Scaffold_01	GenSAS_5d9637f372b5d-publish	CDS	1995	2095	.	+	1	ID=PGEN_.00g000010.m01.CDS02;Name=PGEN_.00g000010.m01.CDS02;Parent=PGEN_.00g000010.m01;original_ID=cds.21510-PGEN_.00g234140.m01;Alias=cds.21510-PGEN_.00g234140.m01
Scaffold_01	GenSAS_5d9637f372b5d-publish	CDS	3325	3495	.	+	0	ID=PGEN_.00g000010.m01.CDS03;Name=PGEN_.00g000010.m01.CDS03;Parent=PGEN_.00g000010.m01;original_ID=cds.21510-PGEN_.00g234140.m01;Alias=cds.21510-PGEN_.00g234140.m01
Scaffold_01	GenSAS_5d9637f372b5d-publish	CDS	4651	4719	.	+	0	ID=PGEN_.00g000010.m01.CDS04;Name=PGEN_.00g000010.m01.CDS04;Parent=PGEN_.00g000010.m01;original_ID=cds.21510-PGEN_.00g234140.m01;Alias=cds.21510-PGEN_.0

In [20]:
%%bash
files_array=(Panopea-generosa-vv0.74.a4.CDS.gff3 Panopea-generosa-vv0.74.a4.exon.gff3 Panopea-generosa-vv0.74.a4.gene.gff3 Panopea-generosa-vv0.74.a4.mRNA.gff3 Panopea-generosa-vv0.74.a4.repeat_region.gff3 Panopea-generosa-vv0.74.a4.rRNA.gff3 Panopea-generosa-vv0.74.a4.tRNA.gff3 Pgenerosa_v074.fa Pgenerosa_v074.fa.fai Pgenerosa_v074.CpG.gff Panopea-generosa-vv0.74.a4.intergenic.bed Panopea-generosa-vv0.74.a4.introns.bed Panopea-generosa-vv0.74.a4.repeats.DNA.gff3 Panopea-generosa-vv0.74.a4.repeats.LTR.gff3 Panopea-generosa-vv0.74.a4.repeats.SINE.gff3 Panopea-generosa-vv0.74.a4.repeats.Unknown.gff3 Panopea-generosa-vv0.74.a4.repeats.LINE.gff3 Panopea-generosa-vv0.74.a4.repeats.RC.gff3 Panopea-generosa-vv0.74.a4.repeats.Simple_repeat.gff3)

for file in "${files_array[@]}"
do
  rm "${file}"
done

ls -ltrh

total 3.7G
-rw-rw-r-- 1 sam users  53M Nov  5 08:55 Panopea-generosa-v1.0.a4.CDS.gff3
-rw-rw-r-- 1 sam users  55M Nov  5 08:55 Panopea-generosa-v1.0.a4.exon.gff3
-rw-rw-r-- 1 sam users 9.5M Nov  5 08:55 Panopea-generosa-v1.0.a4.gene.gff3
-rw-rw-r-- 1 sam users 9.1M Nov  5 08:55 Panopea-generosa-v1.0.a4.mRNA.gff3
-rw-rw-r-- 1 sam users 326M Nov  5 08:55 Panopea-generosa-v1.0.a4.repeat_region.gff3
-rw-rw-r-- 1 sam users 1.2K Nov  5 08:55 Panopea-generosa-v1.0.a4.rRNA.gff3
-rw-rw-r-- 1 sam users 2.3M Nov  5 08:55 Panopea-generosa-v1.0.a4.tRNA.gff3
-rw-rw-r-- 1 sam users 914M Nov  5 08:56 Panopea-generosa-v1.0.fa
-rw-rw-r-- 1 sam users  658 Nov  5 08:56 Panopea-generosa-v1.0.fa.fai
-rw-rw-r-- 1 sam users 2.0G Nov  5 08:59 Panopea-generosa-v1.0.CpG.gff
-rw-rw-r-- 1 sam users 996K Nov  5 08:59 Panopea-generosa-v1.0.a4.intergenic.bed
-rw-rw-r-- 1 sam users 4.4M Nov  5 08:59 Panopea-generosa-v1.0.a4.introns.bed
-rw-rw-r-- 1 sam users 4.7M Nov  5 08:59 Panopea-generosa-v1.0.a4.repeats.DNA.gff3
